In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
import math

In [3]:
# ==============================================================================
# SPARK SESSION INITIALIZATION
# ==============================================================================

print("=" * 80)
print("INITIALIZING SPARK SESSION")
print("=" * 80)

spark = SparkSession.builder \
    .appName("TaxiDataProcessing") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.parquet.mergeSchema", "true") \
    .config("spark.sql.files.maxPartitionBytes", "128MB") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

INITIALIZING SPARK SESSION


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/24 18:22:36 WARN Utils: Your hostname, Kaweewat.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.100 instead (on interface en0)
25/11/24 18:22:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/24 18:22:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# ==============================================================================
# DATA LOADING WITH SAMPLING (Memory-Safe Daily File Processing)
# ==============================================================================

print("\n" + "=" * 80)
print("LOADING DATA FROM DAILY PARQUET FILES WITH SAMPLING")
print("=" * 80)

import glob
import os
from datetime import datetime

# Configuration
SAMPLE_FRACTION = 0.10  # 10% sample
BANGKOK_TIMEZONE = "Asia/Bangkok"  # GMT+7
MIN_DATE = "2021-01-01"  # Filter out invalid dates (include 2021 onwards)

# Base paths containing Hive partitioned data
base_paths = ["PARQUET-PROBE-202301", "PARQUET-PROBE-202302"]

# Collect all daily partitions
all_daily_partitions = []

for base_path in base_paths:
    if os.path.exists(base_path):
        # Find all date=YYYY-MM-DD subdirectories
        date_partitions = glob.glob(f"{base_path}/date=*")
        
        # Filter out invalid dates (before 2023-01-01)
        for partition in date_partitions:
            # Extract date from partition name
            date_str = partition.split("date=")[-1]
            try:
                # Validate date format and filter
                partition_date = datetime.strptime(date_str, "%Y-%m-%d")
                if partition_date >= datetime.strptime(MIN_DATE, "%Y-%m-%d"):
                    all_daily_partitions.append(partition)
            except ValueError:
                print(f"   ⚠ Skipping invalid date partition: {partition}")
                continue

# Sort partitions by date
all_daily_partitions.sort()

print(f"\nFound {len(all_daily_partitions)} valid daily partitions (from {MIN_DATE} onwards)")
print(f"Sample fraction: {SAMPLE_FRACTION * 100}%")
print(f"Date range: {all_daily_partitions[0].split('=')[-1]} to {all_daily_partitions[-1].split('=')[-1]}")

if not all_daily_partitions:
    raise ValueError(f"No valid partitions found after {MIN_DATE}!")

# Process partitions one by one and accumulate samples
sampled_dfs = []
total_records_processed = 0
total_records_sampled = 0
failed_partitions = []

for idx, partition_path in enumerate(all_daily_partitions, 1):
    try:
        partition_date = partition_path.split("date=")[-1]
        print(f"\n[{idx}/{len(all_daily_partitions)}] Processing: {partition_date}")
        
        # Load single daily partition
        df_daily = spark.read.parquet(partition_path)
        
        # Get record count (fast with parquet metadata)
        daily_count = df_daily.count()
        
        if daily_count == 0:
            print(f"   ⚠ Empty partition, skipping...")
            continue
            
        print(f"   Records: {daily_count:,}")
        
        # Take 10% sample with fixed seed for reproducibility
        df_sample = df_daily.sample(withReplacement=False, fraction=SAMPLE_FRACTION, seed=42)
        
        # Convert timestamp to Bangkok timezone (GMT+7)
        print(f"   Converting timestamps to Bangkok timezone...")
        
        # Handle different timestamp formats
        df_sample = df_sample.withColumn("timestamp_raw", F.col("timestamp"))
        
        # Convert to timestamp type
        df_sample = df_sample.withColumn("timestamp",
            # If Unix epoch in seconds (< 10 billion)
            F.when(F.col("timestamp").cast("long") < 10000000000,
                   F.from_unixtime(F.col("timestamp").cast("long")))
            # If Unix epoch in milliseconds
            .when(F.col("timestamp").cast("long") >= 10000000000,
                  F.from_unixtime(F.col("timestamp").cast("long") / 1000))
            # If already timestamp/string
            .otherwise(F.to_timestamp(F.col("timestamp")))
        )
        
        # Ensure proper timestamp type
        df_sample = df_sample.withColumn("timestamp",
            F.to_timestamp(F.col("timestamp"))
        )
        
        # Convert from UTC to Bangkok timezone (GMT+7)
        df_sample = df_sample.withColumn("timestamp",
            F.from_utc_timestamp(F.col("timestamp"), BANGKOK_TIMEZONE)
        )
        
        # Drop temporary column
        df_sample = df_sample.drop("timestamp_raw")
        
        # Get sample count
        sample_count = df_sample.count()
        print(f"   Sampled: {sample_count:,} records")
        
        # Add to collection
        sampled_dfs.append(df_sample)
        
        # Immediately free memory from full daily dataframe
        df_daily.unpersist()
        del df_daily
        
        # Update totals
        total_records_processed += daily_count
        total_records_sampled += sample_count
        
        print(f"   ✓ Completed. Memory freed.")
        
        # Periodic garbage collection every 10 partitions
        if idx % 10 == 0:
            import gc
            gc.collect()
            print(f"   🔄 Garbage collection triggered")
        
    except Exception as e:
        print(f"   ❌ Error processing {partition_path}: {str(e)}")
        failed_partitions.append(partition_path)
        continue

# Combine all sampled dataframes
print(f"\n{'='*80}")
print("COMBINING SAMPLED DATA")
print(f"{'='*80}")

if not sampled_dfs:
    raise ValueError("No data was loaded successfully!")

print(f"\nCombining {len(sampled_dfs)} sampled partitions...")

# Union all samples efficiently
df = sampled_dfs[0]
for i, sample_df in enumerate(sampled_dfs[1:], 2):
    df = df.union(sample_df)
    if i % 20 == 0:
        print(f"   Combined {i}/{len(sampled_dfs)} partitions...")

# Free memory from individual samples
for sample_df in sampled_dfs:
    sample_df.unpersist()
del sampled_dfs

print(f"\n{'='*80}")
print("LOADING SUMMARY")
print(f"{'='*80}")
print(f"✓ Partitions processed: {len(all_daily_partitions)}")
print(f"✓ Partitions loaded: {len(all_daily_partitions) - len(failed_partitions)}")
if failed_partitions:
    print(f"⚠ Partitions failed: {len(failed_partitions)}")
print(f"✓ Total records processed: {total_records_processed:,}")
print(f"✓ Total records sampled: {total_records_sampled:,}")
print(f"✓ Effective sample rate: {(total_records_sampled/total_records_processed*100):.2f}%")

print(f"\n--- Schema ---")
df.printSchema()

print("\n--- Sample Data (with Bangkok timezone) ---")
df.select("vehicle_id", "timestamp", "lat", "lon", "speed").show(10, truncate=False)

print(f"\n--- Timestamp Range (Bangkok Time) ---")
df.select(
    F.min("timestamp").alias("earliest_timestamp"),
    F.max("timestamp").alias("latest_timestamp"),
    F.count("*").alias("total_records")
).show(truncate=False)

print(f"\n--- Date Distribution ---")
df.groupBy(F.to_date("timestamp").alias("date")) \
    .count() \
    .orderBy("date") \
    .show(20, truncate=False)

print(f"Initial record count: {df.count():,}")


LOADING DATA FROM DAILY PARQUET FILES WITH SAMPLING

Found 162 valid daily partitions (from 2021-01-01 onwards)
Sample fraction: 10.0%
Date range: 2021-08-04 to 2023-02-28

[1/162] Processing: 2021-08-04


   Records: 1
   Converting timestamps to Bangkok timezone...
   Sampled: 0 records
   ✓ Completed. Memory freed.

[2/162] Processing: 2022-02-04
   Records: 8
   Converting timestamps to Bangkok timezone...
   Sampled: 1 records
   ✓ Completed. Memory freed.

[3/162] Processing: 2022-02-23
   Records: 1
   Converting timestamps to Bangkok timezone...
   Sampled: 0 records
   ✓ Completed. Memory freed.

[4/162] Processing: 2022-03-08
   Records: 74
   Converting timestamps to Bangkok timezone...
   Sampled: 7 records
   ✓ Completed. Memory freed.

[5/162] Processing: 2022-03-12
   Records: 9
   Converting timestamps to Bangkok timezone...
   Sampled: 1 records
   ✓ Completed. Memory freed.

[6/162] Processing: 2022-03-16
   Records: 60
   Converting timestamps to Bangkok timezone...
   Sampled: 6 records
   ✓ Completed. Memory freed.

[7/162] Processing: 2022-04-13
   Records: 85
   Converting timestamps to Bangkok timezone...
   Sampled: 8 records
   ✓ Completed. Memory freed.

[8/162

25/11/24 18:24:51 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
25/11/24 18:24:51 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
25/11/24 18:24:51 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB


+---------------------------+-------------------+--------+---------+-----+
|vehicle_id                 |timestamp          |lat     |lon      |speed|
+---------------------------+-------------------+--------+---------+-----+
|ceA0iCVUC5bYCXyW+BpVQ90cAOY|2022-02-04 23:41:55|13.57436|100.81136|0.0  |
|cBkMk2JO+U1KOHdagHkqb8SsHTY|2022-03-08 21:34:19|13.60358|100.70549|0.0  |
|cBkMk2JO+U1KOHdagHkqb8SsHTY|2022-03-08 21:34:19|13.60358|100.70549|0.0  |
|cBkMk2JO+U1KOHdagHkqb8SsHTY|2022-03-08 21:34:19|13.60358|100.70549|0.0  |
|cBkMk2JO+U1KOHdagHkqb8SsHTY|2022-03-08 21:34:19|13.60358|100.70549|0.0  |
|cBkMk2JO+U1KOHdagHkqb8SsHTY|2022-03-08 21:34:19|13.60358|100.70549|0.0  |
|cBkMk2JO+U1KOHdagHkqb8SsHTY|2022-03-08 21:34:19|13.60358|100.70549|0.0  |
|cBkMk2JO+U1KOHdagHkqb8SsHTY|2022-03-08 21:34:19|13.60358|100.70549|0.0  |
|BjAsxpgR/cqNmc0t7rFPF4cuNG0|2022-03-12 14:53:09|13.76253|100.76036|0.0  |
|bxpVwzv8diD+rzPxReeP9/eMm+o|2022-03-16 19:54:31|13.64605|100.59546|0.0  |
+------------------------

25/11/24 18:24:54 WARN DAGScheduler: Broadcasting large task binary with size 1688.2 KiB


+-------------------+-------------------+-------------+
|earliest_timestamp |latest_timestamp   |total_records|
+-------------------+-------------------+-------------+
|2022-02-04 23:41:55|2023-03-01 06:59:13|10548553     |
+-------------------+-------------------+-------------+


--- Date Distribution ---


25/11/24 18:25:01 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
25/11/24 18:25:10 WARN DAGScheduler: Broadcasting large task binary with size 1484.9 KiB


+----------+-----+
|date      |count|
+----------+-----+
|2022-02-04|1    |
|2022-03-08|7    |
|2022-03-12|1    |
|2022-03-16|10   |
|2022-04-13|15   |
|2022-04-17|4    |
|2022-05-18|4    |
|2022-06-14|7    |
|2022-06-24|10   |
|2022-08-07|7    |
|2022-09-27|10   |
|2022-11-04|1    |
|2022-11-11|1    |
|2022-11-15|1    |
|2022-12-09|1    |
|2022-12-14|2    |
|2022-12-20|1    |
|2022-12-23|1    |
|2022-12-24|1    |
|2022-12-25|1    |
+----------+-----+
only showing top 20 rows


Initial record count: 10,548,553


### Data Cleaning

In [6]:
# ==============================================================================
# DATA CLEANSING
# ==============================================================================

print("\n" + "=" * 80)
print("STARTING DATA CLEANSING")
print("=" * 80)

# ==============================================================================
# OPTIMIZED DATA CLEANSING (Single Pass)
# ==============================================================================

print("\n" + "=" * 80)
print("STARTING OPTIMIZED DATA CLEANSING")
print("=" * 80)

# OPTIMIZATION 1: Combine ALL filters and transformations in ONE pass
print("\n1. Applying all filters and basic transformations in single pass...")

df = df.filter(
    # Timestamp filter - include 2021 onwards
    (F.col("timestamp") >= "2021-01-01") &
    # GPS validity
    ((F.col("gpsvalid") == 1) | ((F.col("lat") != 0.0) & (F.col("lon") != 0.0))) &
    # Thailand bounds
    (F.col("lat").between(5.6, 20.5)) & (F.col("lon").between(97.3, 105.6))
).select(
    # Keep only necessary columns, drop unused ones early
    "vehicle_id",
    "timestamp",
    "lat",
    "lon",
    F.when(F.col("speed") > 150, 150)
     .when(F.col("speed") < 0, 0)
     .otherwise(F.coalesce(F.col("speed"), F.lit(0))).alias("speed"),
    F.coalesce(F.col("heading"), F.lit(0)).alias("heading"),
    "for_hire_light",
    "engine_acc",
    "gpsvalid"
).dropDuplicates(["vehicle_id", "timestamp", "lat", "lon"])

print(f"✓ Filtering and deduplication complete")


STARTING DATA CLEANSING

STARTING OPTIMIZED DATA CLEANSING

1. Applying all filters and basic transformations in single pass...
✓ Filtering and deduplication complete


### Feature Engineering

In [7]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from typing import Dict, Tuple

# ==============================================================================
# OPTIMIZED FEATURE ENGINEERING (Batch Operations)
# ==============================================================================

print("\n" + "=" * 80)
print("STARTING OPTIMIZED FEATURE ENGINEERING")
print("=" * 80)

# Define constants and helper functions
# ---

# Bangkok Metropolitan Region (BMR) Province Centers (lat, lon)
province_centers: Dict[str, Tuple[float, float]] = {
    'bangkok': (13.7563, 100.5018),
    'nakhon_pathom': (13.8199, 100.0622),
    'pathum_thani': (14.0208, 100.5250),
    'nonthaburi': (13.8621, 100.5144),
    'samut_prakan': (13.5990, 100.5998),
    'samut_sakhon': (13.5475, 100.2744)
}

# Haversine distance calculation in km (using Spark SQL functions)
# The formula calculates the great-circle distance between two points on a sphere.
def haversine_distance_km(lat_col, lon_col, center_lat, center_lon) -> F.Column:
    """Calculates the Haversine distance between a point (lat_col, lon_col)
    and a fixed center (center_lat, center_lon) in kilometers."""
    R = 6371  # Earth radius in kilometers
    
    # Calculate angular distance 'c' using the formula
    c = F.acos(
        F.sin(F.radians(lat_col)) * F.sin(F.radians(F.lit(center_lat))) +
        F.cos(F.radians(lat_col)) * F.cos(F.radians(F.lit(center_lat))) *
        F.cos(F.radians(lon_col) - F.radians(F.lit(center_lon)))
    )
    return c * R

# ---

# OPTIMIZATION: Create ALL non-window features in ONE transformation
print("\n1. Creating temporal, spatial, and activity features (single pass)...")

# Bangkok Metropolitan Region Boundaries (approximate)
bmr_lat_min, bmr_lat_max = 13.4, 14.2
bmr_lon_min, bmr_lon_max = 100.0, 100.9

# Create distance columns using the helper function
distance_cols = [
    haversine_distance_km(F.col("lat"), F.col("lon"), lat, lon).alias(f"distance_from_{name}_km")
    for name, (lat, lon) in province_centers.items()
]

df = df.select(
    "*",
    # === TEMPORAL FEATURES ===
    F.hour("timestamp").alias("hour"),
    # 0=Sunday, 1=Monday... 6=Saturday (F.dayofweek() returns 1=Sun, 7=Sat)
    (F.dayofweek("timestamp") - 1).alias("day_of_week"), 
    F.date_format("timestamp", "EEEE").alias("day_name"),
    # FIXED: is_weekend now correctly flags F.dayofweek() 1 (Sunday) and 7 (Saturday)
    F.when(F.dayofweek("timestamp").isin([1, 7]), 1).otherwise(0).alias("is_weekend"),
    F.month("timestamp").alias("month"),
    F.year("timestamp").alias("year"),
    F.to_date("timestamp").alias("date_only"),
    
    # Time of day (More detailed, supports various persona needs)
    F.when(F.hour("timestamp").between(5, 11), "morning")    # 05:00 - 11:59
     .when(F.hour("timestamp").between(12, 16), "afternoon") # 12:00 - 16:59
     .when(F.hour("timestamp").between(17, 20), "evening")   # 17:00 - 20:59
     .otherwise("night").alias("time_of_day"),               # 21:00 - 04:59
    
    # Generic Day/Night Indicator (Crucial for Personas)
    # Day Shift (e.g., 06:00-17:59) vs Night Shift (e.g., 18:00-05:59)
    F.when(F.hour("timestamp").between(6, 17), "day_shift")
      .otherwise("night_shift").alias("day_night_shift"),
    
    # Rush hour (aligned for high traffic/demand)
    F.when(F.hour("timestamp").isin([7, 8, 9, 17, 18, 19]), 1).otherwise(0).alias("is_rush_hour"),
    
    # === VEHICLE ACTIVITY FEATURES (Supports TK & Nipun) ===
    # Trip-Defining Logic (for Taxi Fleet Manager TK)
    # Assumes 'for_hire_light' == 0 AND 'engine_acc' == 1 means HIRED/OCCUPIED.
    F.when((F.col("for_hire_light") == 0) & (F.col("engine_acc") == 1), 1).otherwise(0).alias("is_hired"),
    
    # Proxy for Searching for Passenger (Taxi Deserts for City Planner Chana)
    # 'for_hire_light' == 1 (light ON) AND 'engine_acc' == 1 means VACANT and MOVING.
    F.when((F.col("for_hire_light") == 1) & (F.col("engine_acc") == 1), 1).otherwise(0).alias("is_searching_fare"),
    
    # Idle/Stationary Status
    F.when(F.col("engine_acc") == 0, 1).otherwise(0).alias("is_idle"),
    F.when(F.col("speed") > 3, 1).otherwise(0).alias("is_moving"),
    F.when(F.col("speed") <= 3, 1).otherwise(0).alias("is_stationary"),
    
    # Speed category
    F.when(F.col("speed") <= 3, "stationary")
      .when(F.col("speed") <= 30, "slow")
      .when(F.col("speed") <= 60, "moderate")
      .when(F.col("speed") <= 90, "fast")
      .otherwise("very_fast").alias("speed_category"),
    
    # === SPATIAL FEATURES - Bangkok Metropolitan Region ===
    # Apply the Haversine distance columns
    *distance_cols,
    
    # Flag for Bangkok Metropolitan Region
    F.when(
        (F.col("lat").between(bmr_lat_min, bmr_lat_max)) &
        (F.col("lon").between(bmr_lon_min, bmr_lon_max)),
        1
    ).otherwise(0).alias("is_in_bmr"),
    
    # Grid cells (for hotspot analysis - Chana)
    (F.round(F.col("lat") * 100) / 100).alias("lat_grid"), # 100 = ~1.1km grid
    (F.round(F.col("lon") * 100) / 100).alias("lon_grid")
)

# 🚨 NEW FILTER STEP ADDED HERE 🚨
# Filter the DataFrame to keep only records within the BMR.
print("1b. Filtering to keep only data within Bangkok Metropolitan Region (BMR)...")
df = df.filter(F.col("is_in_bmr") == 1)
print("✓ Filter applied. Data outside BMR has been removed.")

# ---
print("2. Calculating derived spatial features and adding tourist proxies...")

# Create a list of distance column names
distance_col_names = [f"distance_from_{name}_km" for name in province_centers.keys()]

# Add derived spatial features (depends on previous calculations)
df = df.withColumn("distance_to_nearest_center_km",
    F.least(*[F.col(c) for c in distance_col_names])
)

# Identify nearest province center 
nearest_province_expression = None
province_names = list(province_centers.keys())

for name in province_names:
    col_name = f"distance_from_{name}_km"
    province_display_name = name.replace('_', ' ').title()
    
    condition = (F.col("distance_to_nearest_center_km") == F.col(col_name))
    
    if nearest_province_expression is None:
        nearest_province_expression = F.when(condition, province_display_name)
    else:
        # Use otherwise to chain the conditions
        nearest_province_expression = nearest_province_expression.when(condition, province_display_name)

# Finalize the expression with an otherwise clause
# Since we filtered out data where is_in_bmr was 0, the 'Outside BMR' clause here is theoretically only for edge cases 
# or where the BMR coordinates were slightly different from the province center definitions.
df = df.withColumn("nearest_province",
    nearest_province_expression.otherwise("Outside BMR/Edge Case")
)

# Area classification based on distance from Bangkok center
df = df.withColumn("area_type",
    F.when(F.col("distance_from_bangkok_km") <= 5, "bangkok_center_core") # <= 5km
      .when(F.col("distance_from_bangkok_km") <= 15, "bangkok_inner_ring") # 5-15km
      .when(F.col("distance_from_bangkok_km") <= 30, "bangkok_outer_belt") # 15-30km
      # The check F.col("is_in_bmr") == 1 is now redundant since we filtered the data
      .otherwise("bmr_suburban_area") # If not in the rings, but still in BMR
)

# Add grid cell identifier
df = df.withColumn("grid_cell", 
    F.concat_ws("_", F.col("lat_grid"), F.col("lon_grid"))
)


# === NEW TOURIST PROXY FEATURES (Calculated only on BMR data) ===
# --- Bangkok's Major Tourist Hubs and Airports (Approximate lat/lon) ---
# Suvarnabhumi Airport (BKK)
BKK_lat, BKK_lon = 13.6900, 100.7501 
# Don Mueang Airport (DMK)
DMK_lat, DMK_lon = 13.9138, 100.6015 
# Grand Palace / Old City Center (Major Tourist Area)
PALACE_lat, PALACE_lon = 13.7500, 100.4917 
# Chatuchak Weekend Market
JJ_lat, JJ_lon = 13.8037, 100.5516

# Calculate distance to major destinations
df = df.withColumn("distance_to_bkk_airport_km",
    haversine_distance_km(F.col("lat"), F.col("lon"), BKK_lat, BKK_lon)
).withColumn("distance_to_dmk_airport_km",
    haversine_distance_km(F.col("lat"), F.col("lon"), DMK_lat, DMK_lon)
).withColumn("distance_to_grand_palace_km",
    haversine_distance_km(F.col("lat"), F.col("lon"), PALACE_lat, PALACE_lon)
).withColumn("distance_to_chatuchak_km",
    haversine_distance_km(F.col("lat"), F.col("lon"), JJ_lat, JJ_lon)
)

# Feature 1: Proximity to Key Tourist Hubs/Airports
# A point is considered a 'tourism destination' if it's close to EITHER of the major areas.
# Thresholds: 5km for Airports (pick-up/drop-off), 2km for Palace/Chatuchak (dense activity).
df = df.withColumn("is_near_tourist_hub",
    F.when(
        (F.col("distance_to_bkk_airport_km") <= 5) |
        (F.col("distance_to_dmk_airport_km") <= 5) |
        (F.col("distance_to_grand_palace_km") <= 2) |
        (F.col("distance_to_chatuchak_km") <= 2),
        1
    ).otherwise(0)
)

# Feature 2: Time Window for Tourist Activity
# Standard tourist hours: 9am-6pm (excluding the early-morning workday commute, 7-9am).
# We'll use day_of_week (0=Sun, 6=Sat) and hour.
df = df.withColumn("is_tourist_time",
    F.when(
        # Weekend
        (F.col("day_of_week").isin([0, 6])) | 
        # Weekday during mid-day (after morning rush, before evening rush)
        ((F.col("day_of_week").between(1, 5)) & (F.col("hour").between(10, 16))), 
        1
    ).otherwise(0)
)


# Feature 3: FINAL Proxy - High Possibility of Tourist
# Combines: Near a Hub AND It's Tourist Time AND NOT Rush Hour
# This creates a strong signal for non-commuter activity near tourist areas.
df = df.withColumn("is_tourist_activity_proxy",
    F.when(
        (F.col("is_near_tourist_hub") == 1) & 
        (F.col("is_tourist_time") == 1) &
        (F.col("is_rush_hour") == 0) &
        (F.col("is_hired") == 1), # Only consider if the taxi is actually hired/occupied
        1
    ).otherwise(0)
)

# Feature 4: Taxi Pick-up/Drop-off near Airports (Crucial for demand analysis)
# Flagging records that are very close to an airport AND the taxi is hired/occupied.
df = df.withColumn("is_airport_fare_area",
    F.when(
        (F.col("is_hired") == 1) &
        ((F.col("distance_to_bkk_airport_km") <= 1) | (F.col("distance_to_dmk_airport_km") <= 1)), # Very close proximity (1km)
        1
    ).otherwise(0)
)


# ---
print("✓ Optimized temporal, spatial, activity, and tourist features created successfully.")
print("=" * 80)
# ---


STARTING OPTIMIZED FEATURE ENGINEERING

1. Creating temporal, spatial, and activity features (single pass)...
1b. Filtering to keep only data within Bangkok Metropolitan Region (BMR)...
✓ Filter applied. Data outside BMR has been removed.
2. Calculating derived spatial features and adding tourist proxies...
✓ Optimized temporal, spatial, activity, and tourist features created successfully.


In [8]:
# OPTIMIZATION 3: Efficient repartitioning for window operations
print("\n2. Optimizing data layout for window operations...")
num_vehicles = df.select("vehicle_id").distinct().count()
optimal_partitions = min(200, max(50, int(num_vehicles ** 0.5)))
print(f"   Repartitioning to {optimal_partitions} partitions by vehicle_id...")

df = df.repartition(optimal_partitions, "vehicle_id").sortWithinPartitions("vehicle_id", "timestamp")
df.cache()
df.count()  # Materialize cache
print("✓ Data repartitioned and cached")



2. Optimizing data layout for window operations...


25/11/24 18:25:41 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
25/11/24 18:25:53 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB


   Repartitioning to 57 partitions by vehicle_id...


25/11/24 18:25:53 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/11/24 18:25:55 WARN DAGScheduler: Broadcasting large task binary with size 5.4 MiB
25/11/24 18:26:13 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB
25/11/24 18:26:43 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
25/11/24 18:26:54 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB


✓ Data repartitioned and cached


In [9]:
# OPTIMIZATION 4: Create ALL window-based features in ONE pass
print("\n3. Creating window-based features (optimized single pass)...")

window_lag = Window.partitionBy("vehicle_id").orderBy("timestamp")

df = df.withColumn("prev_lat", F.lag("lat").over(window_lag)) \
    .withColumn("prev_lon", F.lag("lon").over(window_lag)) \
    .withColumn("prev_for_hire", F.lag("for_hire_light").over(window_lag)) \
    .withColumn("time_diff_seconds", 
        (F.col("timestamp").cast("long") - F.lag("timestamp").over(window_lag).cast("long")))

# Calculate distance traveled (vectorized Haversine)
df = df.withColumn("distance_traveled_km",
    F.when(F.col("prev_lat").isNotNull(),
        F.acos(
            F.sin(F.radians(F.col("prev_lat"))) * F.sin(F.radians(F.col("lat"))) +
            F.cos(F.radians(F.col("prev_lat"))) * F.cos(F.radians(F.col("lat"))) *
            F.cos(F.radians(F.col("lon")) - F.radians(F.col("prev_lon")))
        ) * 6371
    ).otherwise(0)
)

# Calculated speed, trip markers, cardinal direction - all in one pass
df = df.select(
    "*",
    # Calculated speed
    F.when(F.col("time_diff_seconds") > 0,
           (F.col("distance_traveled_km") / F.col("time_diff_seconds")) * 3600)
     .otherwise(0).alias("calculated_speed_kmh"),
    
    # Trip start/end
    F.when((F.col("prev_for_hire") == 1) & (F.col("for_hire_light") == 0), 1).otherwise(0).alias("trip_start"),
    F.when((F.col("prev_for_hire") == 0) & (F.col("for_hire_light") == 1), 1).otherwise(0).alias("trip_end"),
    
    # Cardinal direction (simplified)
    F.when(F.col("heading").between(337.5, 360) | F.col("heading").between(0, 22.5), "N")
     .when(F.col("heading").between(22.5, 67.5), "NE")
     .when(F.col("heading").between(67.5, 112.5), "E")
     .when(F.col("heading").between(112.5, 157.5), "SE")
     .when(F.col("heading").between(157.5, 202.5), "S")
     .when(F.col("heading").between(202.5, 247.5), "SW")
     .when(F.col("heading").between(247.5, 292.5), "W")
     .when(F.col("heading").between(292.5, 337.5), "NW")
     .otherwise("N").alias("cardinal_direction"),
    
    # Stop duration and quality flags
    F.when((F.col("is_stationary") == 1) & F.col("time_diff_seconds").isNotNull(),
           F.col("time_diff_seconds")).otherwise(0).alias("stop_duration"),
    
    F.when(F.col("engine_acc") == 1, 1).otherwise(3).alias("normal_frequency")
)

# Add derived behavioral features
df = df.withColumn("is_long_stop", F.when(F.col("stop_duration") > 300, 1).otherwise(0)) \
    .withColumn("data_quality_flag",
        F.when(F.abs((F.col("time_diff_seconds") / 60) - F.col("normal_frequency")) > 2, "irregular")
         .otherwise("normal")) \
    .withColumn("speed_validation_flag",
        F.when((F.col("calculated_speed_kmh").isNotNull()) & 
               (F.abs(F.col("calculated_speed_kmh") - F.col("speed")) > 20), "mismatch")
         .otherwise("ok"))

print("✓ Window-based features created")


3. Creating window-based features (optimized single pass)...
✓ Window-based features created


In [10]:
# ==============================================================================
# OPTIMIZATION 5: Daily aggregations with single groupBy
# ==============================================================================

print("\n4. Creating daily aggregated features (optimized)...")

# --- CORRECTIONS & ENHANCEMENTS ---
# 1. 'likely_has_passenger' is replaced by 'is_hired'.
# 2. 'likely_vacant' is replaced by 'is_searching_fare' (more specific).
# 3. Added 'is_idle' and 'is_stationary' sums for operational insight (TK/Chana).
# 4. Added count of Day/Night shifts for shift planning (TK/Nipun).
# 5. Assume 'distance_traveled_km', 'trip_start', and 'trip_end' are available 
#    from a prior window function (Optimization 3/4).
# ----------------------------------

daily_stats = df.groupBy("vehicle_id", "date_only").agg(
    F.count("*").alias("daily_ping_count"),
    
    # === Vehicle Performance ===
    F.mean("speed").alias("daily_avg_speed"),
    F.max("speed").alias("daily_max_speed"),
    F.sum("distance_traveled_km").alias("daily_total_distance_km"),
    
    # === Activity Counts (TK/Nipun/Chana) ===
    # Total time/pings spent hired (Hired Revenue/Utilization)
    F.sum("is_hired").alias("daily_hired_pings"),
    # Total time/pings spent searching for fares (Empty Miles Proxy)
    F.sum("is_searching_fare").alias("daily_searching_fare_pings"), 
    # Total time/pings spent idle (Operational Cost/Waste)
    F.sum("is_idle").alias("daily_idle_pings"),
    F.sum("is_stationary").alias("daily_stationary_pings"),
    F.sum("is_moving").alias("daily_moving_pings"),
    
    # === Trip Counts (TK/Chana) ===
    F.sum("trip_start").alias("daily_trip_starts"),
    F.sum("trip_end").alias("daily_trip_ends"),
    
    # === Shift Metrics (Nipun/TK) ===
    # Count of pings during Day vs. Night Shift for capacity analysis
    F.sum(F.when(F.col("day_night_shift") == "day_shift", 1).otherwise(0)).alias("daily_day_shift_pings"),
    F.sum(F.when(F.col("day_night_shift") == "night_shift", 1).otherwise(0)).alias("daily_night_shift_pings"),
    
    # === Spatial Metrics ===
    # Total pings within the defined BMR boundaries
    F.sum("is_in_bmr").alias("daily_bmr_pings")
)

# Broadcast join for small aggregation table (Standard practice)
df = df.join(F.broadcast(daily_stats), on=["vehicle_id", "date_only"], how="left")


# Add derived daily metrics
# ---

# Add utilization rate (Supports TK's Revenue/Fleet Management)
df = df.withColumn("daily_utilization_rate",
    F.when(F.col("daily_ping_count") > 0,
           F.col("daily_hired_pings") / F.col("daily_ping_count"))
     .otherwise(0))

# Add empty cruising rate (Supports TK's Operational Cost Analysis)
df = df.withColumn("daily_empty_rate",
    F.when(F.col("daily_moving_pings") > 0,
           F.col("daily_searching_fare_pings") / F.col("daily_moving_pings"))
     .otherwise(0))

# Add shift preference ratio (Supports TK's Shift Optimization)
df = df.withColumn("daily_night_ratio",
    F.when(F.col("daily_ping_count") > 0,
           F.col("daily_night_shift_pings") / F.col("daily_ping_count"))
     .otherwise(0))


print("✓ Daily aggregated features created and joined (optimized)")


4. Creating daily aggregated features (optimized)...
✓ Daily aggregated features created and joined (optimized)


In [11]:
# OPTIMIZATION 6: Drop temporary columns at the end
print("\n5. Cleaning up...")
df = df.drop("prev_lat", "prev_lon", "prev_for_hire", "normal_frequency")

print("✓ Cleanup complete")


5. Cleaning up...
✓ Cleanup complete


### Summary and download data

In [12]:
# ==============================================================================
# FINAL DATA SUMMARY
# ==============================================================================

print("\n" + "=" * 80)
print("FINAL DATA SUMMARY")
print("=" * 80)

# NOTE: 'num_vehicles' is assumed to be defined elsewhere in the full script
final_count = df.count()
num_vehicles = df.select("vehicle_id").distinct().count() # Recalculated for completeness
print(f"\n✓ Final record count: {final_count:,}")
print(f"✓ Number of columns: {len(df.columns)}")
print(f"✓ Number of unique vehicles: {num_vehicles:,}")

print("\n--- Sample of Processed Data ---")
df.select(
    "vehicle_id", 
    "timestamp", 
    "speed", 
    "area_type", 
    "time_of_day", 
    "is_hired", 
    "daily_utilization_rate"
).show(10, truncate=False)

print("\n" + "=" * 80)
print("DATA PREPARATION COMPLETE - READY FOR ANALYTICS!")
print("=" * 80)


# Save options (choose one based on use case)
print("\n--- Save Options ---")
print("# Option 1: Partitioned Parquet (Recommended for distributed queries)")
df.write.mode("overwrite").partitionBy("year", "month").parquet("taxi_processed.parquet")
# print("\n# Option 2: Single Parquet (Use for small datasets or after coalesce)")
# print('# df.coalesce(10).write.mode("overwrite").parquet("output/taxi_processed.parquet")')
# print("\n# Option 3: Delta Lake (Best for ACID compliance and updates)")
# print('# df.write.format("delta").mode("overwrite").save("output/taxi_processed_delta")')

print("\nProcessed dataframe ready in variable: df")
print("Use df.createOrReplaceTempView('taxi') for SQL queries")

# Unpersist when done
# df.unpersist()


FINAL DATA SUMMARY


25/11/24 18:27:32 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
25/11/24 18:27:33 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB



✓ Final record count: 9,078,919
✓ Number of columns: 73
✓ Number of unique vehicles: 3,282

--- Sample of Processed Data ---


25/11/24 18:27:35 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB
25/11/24 18:27:36 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB


+---------------------------+-------------------+-----+------------------+-----------+--------+----------------------+
|vehicle_id                 |timestamp          |speed|area_type         |time_of_day|is_hired|daily_utilization_rate|
+---------------------------+-------------------+-----+------------------+-----------+--------+----------------------+
|+m8gZf8PGo8Y16q4nEYLlISr+PM|2023-01-04 22:43:47|55.0 |bmr_suburban_area |night      |1       |1.0                   |
|+m8gZf8PGo8Y16q4nEYLlISr+PM|2023-01-04 22:45:40|53.0 |bmr_suburban_area |night      |1       |1.0                   |
|+m8gZf8PGo8Y16q4nEYLlISr+PM|2023-01-04 22:54:40|87.0 |bmr_suburban_area |night      |1       |1.0                   |
|+m8gZf8PGo8Y16q4nEYLlISr+PM|2023-01-04 23:39:58|70.0 |bmr_suburban_area |night      |1       |1.0                   |
|+m8gZf8PGo8Y16q4nEYLlISr+PM|2023-01-05 00:03:58|58.0 |bangkok_outer_belt|night      |1       |0.4659090909090909    |
|+m8gZf8PGo8Y16q4nEYLlISr+PM|2023-01-05 00:06:08

25/11/24 18:27:36 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB
25/11/24 18:27:39 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB



Processed dataframe ready in variable: df
Use df.createOrReplaceTempView('taxi') for SQL queries
